![](img/Tsampl.png) ![](img/Tsar.png)

![](img/ADC%20timing.png)





In [22]:
import numpy as np

#Sample time SMP
Tsampl = np.array([2.5, 6.5, 12.5, 24.5, 47.5, 92.5, 247.5, 640.5])

channel_conf = {"Ibat": {"channel" : 1, "Tsample" : Tsampl[4], "Rin" : 120},
                "Vbat": {"channel" : 8, "Tsample" : Tsampl[4], "Rin" : 120},
                "Vpanel": {"channel" : 3, "Tsample" : Tsampl[4], "Rin" : 120},
                "Ipanel": {"channel" : 5, "Tsample" : Tsampl[4], "Rin" : 120},
                "TempDiode": {"channel" : 9, "Tsample" : Tsampl[3], "Rin" : 1e3},
                "TempMos": {"channel" : 10, "Tsample" : Tsampl[3], "Rin" : 1e3},
                "Rad": {"channel" : 11, "Tsample" : Tsampl[3], "Rin" : 1e3},
                }

# ADC channel sampling order
rank = ["Ibat", "Vbat", "Vpanel", "Ipanel", "TempDiode", "TempMos", "Rad"]

ADC_Res = "12" # ADC Resolution
Tsar = {"12" : 12.5, "10" : 10.5, "8" : 8.5, "6" : 6.5}     # ADC Resolution time SAR

Tadc_clk = 1/80e6 # ADC clock time 1/ADC frequency

Trigger = 7e3  # Trigger frequency

Cadc = 5e-12    #ADC internal capacitor

oversampling_ratio = 32
max_value = oversampling_ratio * ((2**12) -1)
min_bit_shift = max_value.bit_length() - 16
max_value_shifted = max_value >> min_bit_shift

Tconv = []
error = []
for channel in rank:
    Tconv.append(oversampling_ratio * (Tsar[ADC_Res] + channel_conf[channel]["Tsample"]) * Tadc_clk)
    # Vo = 1 - exp(-t/RC)
    error.append((np.exp(-(Tadc_clk * channel_conf[channel]["Tsample"]) / (channel_conf[channel]["Rin"] * Cadc))))


print("Tbussy: ", sum(Tconv))
print("Tfree: ", (1/Trigger) - sum(Tconv))
print("Tfree(%)", 100*((1/Trigger) - sum(Tconv)) / (1/Trigger) )
print("Bit gain", np.log(oversampling_ratio) / np.log(4))
print("maximum value (oversample): ", hex(max_value))
print("maximum value shifted (oversample)", hex(max_value_shifted))

print("========== CONF ==========")
print("Resolution: ", ADC_Res)
print("Oversampling: ", oversampling_ratio, "x")
print("Minimum Bit Shift", min_bit_shift)
print("Number of conversion: ", len(rank))
for channel in rank:
    print("Channel: ",channel)
    print("Channel °N: ", channel_conf[channel]["channel"])
    print("Sampling Time: ", channel_conf[channel]["Tsample"])






Tbussy:  0.0001404
Tfree:  2.4571428571428704e-06
Tfree(%) 1.720000000000009
Bit gain 2.5
maximum value (oversample):  0x1ffe0
maximum value shifted (oversample) 0xfff0
========== CONF ==========
Resolution:  12
Oversampling:  32 x
Minimum Bit Shift 1
Number of conversion:  7
Channel:  Ibat
Channel °N:  1
Sampling Time:  47.5
Channel:  Vbat
Channel °N:  8
Sampling Time:  47.5
Channel:  Vpanel
Channel °N:  3
Sampling Time:  47.5
Channel:  Ipanel
Channel °N:  5
Sampling Time:  47.5
Channel:  TempDiode
Channel °N:  9
Sampling Time:  24.5
Channel:  TempMos
Channel °N:  10
Sampling Time:  24.5
Channel:  Rad
Channel °N:  11
Sampling Time:  24.5
